In [ ]:
# Checklist...
#
# Make sure units have been converted in fisher matrix to match units in mcmc chains (use 'Fisher_Units_Converter' notebook.)
#
# Make sure parameters in fisher matrix match order of parameter in MCMC
#
# Make sure you burn in the right number of points in the MCMC chains before plotting below 
#
# Choose where to plot Fisher contour centers (where you forecasted [correct choice], or max likelihood of MCMCs)
#
#

In [1]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

GetDist Version: 1.0.3, Matplotlib version: 3.1.2


In [ ]:
# First run montepython to determine the burn-in period 

#yesstep_burnin = 
#nostep_burnin = 

In [2]:
#Import MCMC Chains

chains_dir = "/Users/nicholasdeporzio/Desktop/cfworkspace/chains/"
yesstep_dir = chains_dir + "chains_yes_relicfast/chains_only/2019-12-04_1000000_"
nostep_dir = chains_dir + "chains_no_relicfast/chains_only/2019-12-04_1000000_"

samples_yesstep = loadMCSamples(yesstep_dir, settings={'ignore_rows':0.4})
samples_nostep = loadMCSamples(nostep_dir, settings={'ignore_rows':0.4})


In [ ]:
# Triangle plot, no Fisher Contours
g = plots.get_subplot_plotter()
g.triangle_plot(
    [samples_yesstep, samples_nostep], 
    filled=True,
    legend_labels=['MCMC, DESI, Relic/LCDM Steps ON', 'MCMC, DESI, Relic/LCDM Steps OFF'])
plt.savefig("/Users/nicholasdeporzio/Desktop/only_mcmc.png")

In [3]:
# Add Fisher Contours
# Need to adjust Fisher parameter ordering to MCMC parameter ordering
# Fisher parameter ordering: omega_b, omega_cdm, n_s, A_s, tau_reio, h, m_0, sigma_fog, b0_bar, alpha_k2
# MCMC parameter ordering: omega_b, omega_cdm, n_s, A_s, h, tau_reio, m_0, sigma_fog, b0_bar, alpha_k2

fisher_reorder_index = [0, 1, 2, 3, 5, 4, 6, 7, 8, 9]

yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/desi_postshift_RelicStepsOn/inv_fisher.mat", skiprows=1)
nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/desi_preshift_RelicStepsOff_fidsOnMcmcBestfitNoRelicStep/inv_fisher.mat", skiprows=1)

#yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/euclid/forecast_step_on/inv_fisher.mat", skiprows=1)
#nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/euclid/forecast_step_off/inv_fisher.mat", skiprows=1)

yesstep_covariance_out = np.zeros((10,10))
nostep_covariance_out = np.zeros((10,10))

for idx1 in range(10): 
    for idx2 in range(10): 
        yesstep_covariance_out[idx1, idx2] = yesstep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]
        nostep_covariance_out[idx1, idx2] = nostep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]

In [4]:
# Specify fisher fiducials. Note omega_b in units of 10^2 and A_s in units of 10^-9. 
#fisher_fiducials = [2.222, 0.1120, 0.965, 2.22, 0.7, 0.06, 0.03, 250, 1.0, 1.0]

# DESI best fits
#mcmc_yesstep_bestfit = [2.225896, 0.1127521, 0.9659999, 2.234124, 0.7003915, 0.06018715, 0.0365804, 249.5779, 0.9992126, 1.025932]
mcmc_nostep_bestfit = [2.226660, 0.1126965, 0.9662029, 2.231027, 0.7008683, 0.05944537, 0.0364634, 249.6839, 1.004659, 0.9794683]
forecast_yesstep_shift = [
    (2.226660e-00 + 1.092514e-05), 
    (1.126965e-01 + 2.240456e-05),
    (9.662029e-01 - 5.847986e-05),
    (2.231027e-00 - 6.483268e-03),
    (7.008683e-01 + 7.350068e-04),
    (5.944537e-02 - 1.525700e-03),
    (0.0364634 - 0.002999569333),
    (2.496839e+02 + 9.316495e-02),
    (1.004659e+00 - 4.432180e-03),
    (9.794683e-01 + 3.124481e-02)
    ]

# EUCLID best fits
#mcmc_yesstep_bestfit = [2.227193, 1.127846e-01, 9.665308e-01, 2.222313, 7.015911e-01, 5.757700e-02, 0.03387923333, 2.498330e+02, 1.000948e+00, 1.002212e+00]
#mcmc_nostep_bestfit = [2.224951, 1.125220e-01, 9.676774e-01, 2.260311, 6.986288e-01, 6.664679e-02, 0.04654263333, 2.493397e+02,  1.004890e+00, 9.424425e-01]


paramnames = ['omega_b', 'omega_cdm', 'n_s', 'A_s', 'h', 'tau_reio', 'm_0', 'sigma_fog', 'b0bar', 'alpha_k2']
yesstep_gauss = GaussianND(forecast_yesstep_shift, yesstep_covariance_out, names=paramnames)
nostep_gauss = GaussianND(mcmc_nostep_bestfit, nostep_covariance_out, names=paramnames)

In [ ]:
h = plots.get_subplot_plotter()
h.triangle_plot(
    [yesstep_gauss, nostep_gauss], 
    filled=True,
    legend_labels=['Fisher, EUCLID, Relic/LCDM Steps ON', 'Fisher, EUCLID, RELIC/LCDM Steps OFF'])
plt.savefig("/Users/nicholasdeporzio/Desktop/only_fisher.png")

In [5]:
i = plots.get_subplot_plotter()
i.triangle_plot(
    [samples_yesstep, samples_nostep, yesstep_gauss, nostep_gauss], 
    #filled=False,
    legend_labels=['MCMC, DESI, Relic/LCDM Steps ON', 'MCMC, DESI, Relic/LCDM Steps OFF', 'Fisher, DESI, Relic/LCDM Steps ON', 'Fisher, DESI, Relic/LCDM Steps OFF'],
    line_args=[
        {'color':'green'},
        {'color':'darkblue'},
        {'ls':'--', 'lw':2, 'color':'red'},
        {'ls':'--', 'lw':2, 'color':'grey'}], 
    contour_args=[
        {'color':'green', 'filled' : True},
        {'color':'darkblue', 'filled' : True},
        {'ls':'--', 'lw':2, 'color':'red', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'grey', 'filled' : False}],
)
plt.savefig("/Users/nicholasdeporzio/Desktop/mcmc_and_fisher.png")

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

paramnames = ['omega_b', 'omega_cdm', 'n_s', 'A_s', 'tau_reio', 'h', 'm_0', 'sigma_fog', 'b0bar', 'alpha_k2', 'delta_L']
fiducials = [2.222, 0.1120, 0.965, 2.22, 0.06, 0.7, 0.03, 250, 1.0, 1.0, 0.6]
covmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/inv_fisher.mat", skiprows=1)
gauss = GaussianND(fiducials, covmat, names=paramnames)

g = plots.get_subplot_plotter()
g.triangle_plot(
    gauss, 
    filled=True)
plt.savefig("/Users/nicholasdeporzio/Desktop/delta_L_forecast.png")